## CMG_CCS_4D_Seiscmic

Author: Hyunmin Kim | Last Update Date: 2024-05-10

In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os, shutil
from tqdm import tqdm 
from multiprocessing import Pool
import pyvista as pv
from sklearn.preprocessing import MinMaxScaler
from skimage.transform import resize

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable
import time
import argparse
from progress.bar import IncrementalBar

from gan.generator import UnetGenerator
from gan.encoder import Encoder
from gan.discriminator import Discriminator, PatchDiscriminator, ConditionalDiscriminator
from gan.criterion import GeneratorLoss, DiscriminatorLoss, bce_loss
from gan.utils import Logger, initialize_weights
from lib.slack import print_n_send
from lib.seismogram import add_noise_to_seismic
from lib.Sim_CMG_CCS import Sim_CCS
from lib.subfunction import *

### **Step 1**: Import the ensemble data & data preprocessing

In [ ]:
Data = np.load('ensemble_4000.npz')
Porosity = Data['Porosity']
Facies = Data['Facies']
seismic = np.load('ensemble_4000_seismic_total_v3.npy')
ref_num = 17

ground_truth = np.ones((1, 2, 16, 32, 32))
ground_truth[0, 0] = Porosity[ref_num]
ground_truth[0, 1] = Facies[ref_num]
ground_truth_seismic = seismic[ref_num]

In [ ]:
Data = np.load('ensemble_4000.npz')
Porosity = Data['Porosity']
Facies = Data['Facies']
seismic = np.load('ensemble_4000_seismic_total_v3.npy')
ref_num = 17

ground_truth = np.ones((1, 2, 16, 32, 32))
ground_truth[0, 0] = Porosity[ref_num]
ground_truth[0, 1] = Facies[ref_num]

cdf_model = np.ones((100, 2, 16, 32, 32))
for i in range(100):
    cdf_model[i, 0] = Porosity[40*i]
    cdf_model[i, 1] = Facies[40*i]

ensemble = - np.ones((3999, 2, 16, 32, 32))
ensemble[:, 0] = np.delete(Porosity, ref_num, axis=0)
ensemble[:, 1] = np.delete(Facies, ref_num, axis=0)

ground_truth_seismic = seismic[ref_num]
seismic = np.delete(seismic, ref_num, axis=0)

In [ ]:
scaler1 = MinMaxScaler()
seismic[:, 0] = scaler1.fit_transform(seismic[:, 0].reshape(-1, 1)).reshape((3999, 96, 32, 32)) * 2 - 1

scaler2 = MinMaxScaler()
seismic[:, 1] = scaler2.fit_transform(seismic[:, 1].reshape(-1, 1)).reshape((3999, 96, 32, 32)) * 2 - 1

scaler3 = MinMaxScaler()
seismic[:, 2] = scaler3.fit_transform(seismic[:, 2].reshape(-1, 1)).reshape((3999, 96, 32, 32)) * 2 - 1


scaler4 = MinMaxScaler()
ensemble[:, 0] = scaler4.fit_transform(ensemble[:, 0].reshape(-1, 1)).reshape((3999, 16, 32, 32)) * 2 - 1

ensemble_resized = resize(ensemble, (3999, 2, 96, 32, 32), anti_aliasing=False)
ensemble_resized[:, 1] = np.where(ensemble_resized[:, 1] > 0.5, 1, 0)
ensemble_resized[:, 1] = ensemble_resized[:, 1] * 2 -1

### **Step 2**: define BiCycleGAN model & training

In [ ]:
learning_rate = 0.0002
batch_size = 32
epochs = 1000
lambda_VAE = 100; lambda_latent = 0.5; lambda_KL = 0.01

In [ ]:
device = ('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
# models
print('Defining models!')
generator = UnetGenerator().to(device)
discriminator_cVAE = PatchDiscriminator().to(device)
discriminator_cLR = PatchDiscriminator().to(device)
encoder = Encoder(z_dim=8).to(device)
# optimizers
g_optimizer = torch.optim.Adam(generator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
d_cVAE_optimizer = torch.optim.Adam(discriminator_cVAE.parameters(), lr=learning_rate, betas=(0.5, 0.999))
d_cLR_optimizer = torch.optim.Adam(discriminator_cLR.parameters(), lr=learning_rate, betas=(0.5, 0.999))
e_optimizer = torch.optim.Adam(encoder.parameters(), lr=learning_rate, betas=(0.5, 0.999))
# loss functions
g_criterion = GeneratorLoss(alpha=lambda_VAE)
d_criterion = DiscriminatorLoss()

In [ ]:
generator.load_state_dict(torch.load('runs\\generator_1184_bicycle_0719_v2.pt'))
# encoder.load_state_dict(torch.load('runs\\encoder_401_bicycle_0710.pt'))
# discriminator_cVAE.load_state_dict(torch.load('runs\\discriminator_cVAE_401_bicycle_0710.pt'))
# discriminator_cLR.load_state_dict(torch.load('runs\\discriminator_cLR_401_bicycle_0710.pt'))

In [ ]:
inputs  = torch.tensor(seismic.astype(np.float32))
targets = torch.tensor(ensemble_resized.astype(np.float32))

dataset = TensorDataset(inputs, targets)
data_loader = DataLoader(dataset, batch_size, shuffle=True)

In [ ]:
print('Start of training process!')
logger = Logger(filename='training_log')
for epoch in range(epochs):
    ge_loss=0.
    de_loss=0.
    EGe_loss=0.
    Ge_alone_loss=0.
    z_dim = 8
    
    start = time.time()
    bar = IncrementalBar(f'[Epoch {epoch+1}/{epochs}]', max=len(data_loader))
    for x, real in data_loader:
        x = x.to(device)
        real = real.to(device)
        
        batch_n = x.shape[0]
        x1 = x.reshape(batch_n, 3, 96, 32, 32)
        
        mu, log_variance = encoder(real)
        std = torch.exp(log_variance / 2)
        random_z = Variable(torch.randn(batch_n, z_dim).type(torch.cuda.FloatTensor), requires_grad=True)
        encoded_z = (random_z * std) + mu
        
        # Discriminator`s loss
        fake_cVAE = generator(x1, encoded_z).detach()
        fake_pred_cVAE_1, fake_pred_cVAE_2 = discriminator_cVAE(fake_cVAE)
        fake_pred_cVAE = torch.cat([fake_pred_cVAE_1.reshape(batch_n, -1), fake_pred_cVAE_2.reshape(batch_n, -1)], dim=-1)
        real_pred_cVAE_1, real_pred_cVAE_2 = discriminator_cVAE(real)
        real_pred_cVAE = torch.cat([real_pred_cVAE_1.reshape(batch_n, -1), real_pred_cVAE_2.reshape(batch_n, -1)], dim=-1)
        d_loss_cVAE_1 = mse_loss(fake_pred_cVAE, target_index=0)
        d_loss_cVAE_2 = mse_loss(real_pred_cVAE, target_index=1)
 
        random_z = Variable(torch.randn(batch_n, z_dim).type(torch.cuda.FloatTensor))
        fake_cLR = generator(x1, random_z).detach()
        fake_pred_cLR_1, fake_pred_cLR_2 = discriminator_cLR(fake_cLR)
        fake_pred_cLR = torch.cat([fake_pred_cLR_1.reshape(batch_n, -1), fake_pred_cLR_2.reshape(batch_n, -1)], dim=-1)
        real_pred_cLR_1, real_pred_cLR_2 = discriminator_cLR(real)
        real_pred_cLR = torch.cat([real_pred_cLR_1.reshape(batch_n, -1), real_pred_cLR_2.reshape(batch_n, -1)], dim=-1)
        d_loss_cLR_1 = mse_loss(fake_pred_cLR, target_index=0)
        d_loss_cLR_2 = mse_loss(real_pred_cLR, target_index=1)
        d_loss = (d_loss_cVAE_1 + d_loss_cVAE_2 + d_loss_cLR_1 + d_loss_cLR_2)/4
        
        # Discriminator`s params update
        d_cVAE_optimizer.zero_grad()
        d_cLR_optimizer.zero_grad()
        d_loss.backward()
        d_cVAE_optimizer.step()
        d_cLR_optimizer.step()
        
        for _ in range(2):
            # Generator`s loss
            mu, log_variance = encoder(real)
            std = torch.exp(log_variance / 2)
            random_z = Variable(torch.randn(batch_n, z_dim).type(torch.cuda.FloatTensor))
            encoded_z = (random_z * std) + mu
            
            fake_cVAE = generator(x1, encoded_z)
            fake_pred_cVAE_1, fake_pred_cVAE_2 = discriminator_cVAE(fake_cVAE)
            fake_pred_cVAE = torch.cat([fake_pred_cVAE_1.reshape(batch_n, -1), fake_pred_cVAE_2.reshape(batch_n, -1)], dim=-1)
            g_loss_cVAE = mse_loss(fake_pred_cVAE, target_index=1)
            
            random_z = Variable(torch.randn(batch_n, z_dim).type(torch.cuda.FloatTensor), requires_grad=True)
            fake_cLR = generator(x1, random_z).detach()
            fake_pred_cLR_1, fake_pred_cLR_2 = discriminator_cLR(fake_cLR)
            fake_pred_cLR = torch.cat([fake_pred_cLR_1.reshape(batch_n, -1), fake_pred_cLR_2.reshape(batch_n, -1)], dim=-1)
            g_loss_cLR = mse_loss(fake_pred_cLR, target_index=1)
            g_loss = g_loss_cVAE + g_loss_cLR
            
            KL_div = lambda_KL * torch.sum(0.5 * (mu ** 2 + torch.exp(log_variance) - log_variance - 1))
            img_recon_loss = lambda_VAE * torch.mean(torch.abs(fake_cVAE - real))
            EG_loss = g_loss + KL_div + img_recon_loss
            
            d_cVAE_optimizer.zero_grad()
            d_cLR_optimizer.zero_grad()
            g_optimizer.zero_grad()
            e_optimizer.zero_grad()
            EG_loss.backward()
            e_optimizer.step()
            g_optimizer.step()
        
            # Generator's params only update
            mu_, log_variance_ = encoder(fake_cLR)
            z_recon_loss = torch.mean(torch.abs(mu_ - random_z))
            G_alone_loss = lambda_latent * z_recon_loss
            
            d_cVAE_optimizer.zero_grad()
            d_cLR_optimizer.zero_grad()
            g_optimizer.zero_grad()
            e_optimizer.zero_grad()
            G_alone_loss.backward()
            g_optimizer.step()
            
        # add batch losses
        ge_loss += g_loss.item()
        de_loss += d_loss.item()
        EGe_loss += EG_loss.item()
        Ge_alone_loss += G_alone_loss.item()
        bar.next()
        
    bar.finish()  
    # obttain per epoch losses
    g_loss = ge_loss/len(data_loader)
    d_loss = de_loss/len(data_loader)
    EG_loss = EGe_loss/len(data_loader)
    G_alone_loss = Ge_alone_loss/len(data_loader)
    # count timeframe
    end = time.time()
    tm = (end - start)
    logger.add_scalar('generator_loss', g_loss, epoch+1)
    logger.add_scalar('discriminator_loss', d_loss, epoch+1)
    logger.add_scalar('generator_alone_loss', G_alone_loss, epoch+1)
    logger.add_scalar('EG_loss', EG_loss, epoch+1)
    if epoch % 200 == 0:
        logger.save_weights(generator.state_dict(), f'generator_{epoch}_bicycle_0722')
        logger.save_weights(discriminator_cVAE.state_dict(), f'discriminator_cVAE_{epoch}_bicycle_0722')
        logger.save_weights(discriminator_cLR.state_dict(), f'discriminator_cLR_{epoch}_bicycle_0722')
        logger.save_weights(encoder.state_dict(), f'encoder_{epoch}_bicycle_0722')
    if epoch % 20 == 0:
        print_n_send("Bicycle GAN: [Epoch %d/%d] [G loss: %.3f] [D loss: %.3f] [EG loss: %.3f] [G alone loss: %.3f] ETA: %.3fs" % (epoch+1, epochs, g_loss, d_loss, EG_loss, G_alone_loss, tm))
    else:
        print("Bicycle GAN: [Epoch %d/%d] [G loss: %.3f] [D loss: %.3f] [EG loss: %.3f] [G alone loss: %.3f] ETA: %.3fs" % (epoch+1, epochs, g_loss, d_loss, EG_loss, G_alone_loss, tm))
logger.save_weights(generator.state_dict(), f'generator_{epoch+1}_bicycle_0722')
logger.save_weights(discriminator_cVAE.state_dict(), f'discriminator_cVAE_{epoch+1}_bicycle_0722')
logger.save_weights(discriminator_cLR.state_dict(), f'discriminator_cLR_{epoch+1}_bicycle_0722')
logger.save_weights(encoder.state_dict(), f'encoder_{epoch+1}_bicycle_0722')
logger.close()

print('End of training process!')

In [ ]:
print('Start of training process!')
logger = Logger(filename='training_log')
for epoch in range(epochs):
    ge_loss=0.
    de_loss=0.
    EGe_loss=0.
    Ge_alone_loss=0.
    z_dim = 8
    
    start = time.time()
    bar = IncrementalBar(f'[Epoch {epoch+1}/{epochs}]', max=len(data_loader))
    for x, real in data_loader:
        x = x.to(device)
        real = real.to(device)
        
        batch_n = x.shape[0]
        x1 = x.reshape(batch_n, 3, 96, 32, 32)
        
        mu, log_variance = encoder(real)
        std = torch.exp(log_variance / 2)
        random_z = Variable(torch.randn(batch_n, z_dim).type(torch.cuda.FloatTensor), requires_grad=True)
        encoded_z = (random_z * std) + mu
        
        # Discriminator`s loss
        fake_cVAE = generator(x1, encoded_z).detach()
        fake_pred_cVAE = discriminator_cVAE(fake_cVAE)
        real_pred_cVAE = discriminator_cVAE(real)
        d_loss_cVAE = d_criterion(fake_pred_cVAE, real_pred_cVAE)
 
        random_z = Variable(torch.randn(batch_n, z_dim).type(torch.cuda.FloatTensor))
        fake_cLR = generator(x1, random_z).detach()
        fake_pred_cLR = discriminator_cLR(fake_cLR)
        real_pred_cLR = discriminator_cLR(real)
        d_loss_cLR = d_criterion(fake_pred_cLR, real_pred_cLR)
        d_loss = d_loss_cVAE + d_loss_cLR
        
        # Discriminator`s params update
        d_cVAE_optimizer.zero_grad()
        d_cLR_optimizer.zero_grad()
        d_loss.backward()
        d_cVAE_optimizer.step()
        d_cLR_optimizer.step()
        
        # Generator`s loss
        mu, log_variance = encoder(real)
        std = torch.exp(log_variance / 2)
        random_z = Variable(torch.randn(batch_n, z_dim).type(torch.cuda.FloatTensor))
        encoded_z = (random_z * std) + mu
        
        fake_cVAE = generator(x1, encoded_z)
        fake_pred_cVAE = discriminator_cVAE(fake_cVAE)
        g_loss_cVAE = g_criterion(fake_cVAE, real, fake_pred_cVAE)
        
        random_z = Variable(torch.randn(batch_n, z_dim).type(torch.cuda.FloatTensor), requires_grad=True)
        fake_cLR = generator(x1, random_z).detach()
        fake_pred_cLR = discriminator_cLR(fake_cLR)
        g_loss_cLR = g_criterion(fake_cLR, real, fake_pred_cLR)
        g_loss = g_loss_cVAE + g_loss_cLR
        
        # Generator`s params update
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()
        
        # Encoder's params update
        mu_2, log_variance_2 = encoder(real)
        std_2 = torch.exp(log_variance_2 / 2)
        encoded_z_2 = (random_z * std_2) + mu_2
        fake_cVAE_2 = generator(x1, encoded_z_2)
        fake_pred_cVAE_2 = discriminator_cVAE(fake_cVAE_2)
        g_loss_cVAE_2 = g_criterion(fake_cVAE_2, real, fake_pred_cVAE_2)
        
        fake_cLR_2 = generator(x1, random_z).detach()
        fake_pred_cLR_2 = discriminator_cLR(fake_cLR_2)
        g_loss_cLR_2 = g_criterion(fake_cLR_2, real, fake_pred_cLR_2)
        g_loss_2 = g_loss_cVAE_2.clone() + g_loss_cLR_2.clone()
        
        KL_div = lambda_KL * torch.sum(0.5 * (mu_2 ** 2 + torch.exp(log_variance_2) - log_variance_2 - 1))
        img_recon_loss = lambda_VAE * torch.mean(torch.abs(fake_cVAE_2 - real))
        EG_loss = g_loss_2 + KL_div + img_recon_loss
        
        d_cVAE_optimizer.zero_grad()
        d_cLR_optimizer.zero_grad()
        g_optimizer.zero_grad()
        e_optimizer.zero_grad()
        EG_loss.backward()
        e_optimizer.step()
        g_optimizer.step()
        
        # Generator's params only update
        mu_, log_variance_ = encoder(fake_cLR)
        z_recon_loss = torch.mean(torch.abs(mu_ - random_z))
        G_alone_loss = lambda_latent * z_recon_loss
        
        d_cVAE_optimizer.zero_grad()
        d_cLR_optimizer.zero_grad()
        g_optimizer.zero_grad()
        e_optimizer.zero_grad()
        G_alone_loss.backward()
        g_optimizer.step()
        
        # add batch losses
        ge_loss += g_loss.item()
        de_loss += d_loss.item()
        EGe_loss += EG_loss.item()
        Ge_alone_loss += G_alone_loss.item()
        bar.next()
        
    bar.finish()  
    # obttain per epoch losses
    g_loss = ge_loss/len(data_loader)
    d_loss = de_loss/len(data_loader)
    EG_loss = EGe_loss/len(data_loader)
    G_alone_loss = Ge_alone_loss/len(data_loader)
    # count timeframe
    end = time.time()
    tm = (end - start)
    logger.add_scalar('generator_loss', g_loss, epoch+1)
    logger.add_scalar('discriminator_loss', d_loss, epoch+1)
    logger.add_scalar('generator_alone_loss', G_alone_loss, epoch+1)
    logger.add_scalar('EG_loss', EG_loss, epoch+1)
    if epoch % 200 == 0:
        logger.save_weights(generator.state_dict(), f'generator_{epoch+984}_bicycle_0718_old')
        logger.save_weights(discriminator_cVAE.state_dict(), f'discriminator_cVAE_{epoch+984}_bicycle_0718_old')
        logger.save_weights(discriminator_cLR.state_dict(), f'discriminator_cLR_{epoch+984}_bicycle_0718_old')
        logger.save_weights(encoder.state_dict(), f'encoder_{epoch+984}_bicycle_0718_old')
    if epoch % 50 == 0:
        print_n_send("Bicycle GAN: [Epoch %d/%d] [G loss: %.3f] [D loss: %.3f] [EG loss: %.3f] [G alone loss: %.3f] ETA: %.3fs" % (epoch+1, epochs, g_loss, d_loss, EG_loss, G_alone_loss, tm))
    print("Bicycle GAN: [Epoch %d/%d] [G loss: %.3f] [D loss: %.3f] [EG loss: %.3f] [G alone loss: %.3f] ETA: %.3fs" % (epoch+1, epochs, g_loss, d_loss, EG_loss, G_alone_loss, tm))
logger.save_weights(generator.state_dict(), f'generator_{epoch+984+1}_bicycle_0718_old')
logger.save_weights(discriminator_cVAE.state_dict(), f'discriminator_cVAE_{epoch+984+1}_bicycle_0718_old')
logger.save_weights(discriminator_cLR.state_dict(), f'discriminator_cLR_{epoch+984+1}_bicycle_0718_old')
logger.save_weights(encoder.state_dict(), f'encoder_{epoch+984+1}_bicycle_0718_old')
logger.close()

print('End of training process!')

### **Step 3**: Predict the ground truth models

In [ ]:
N_real = 100
ground_truth_pred = np.ones((N_real, 2, 16, 32, 32)) *-9999

ground_truth_seismic_copy = ground_truth_seismic.copy()
ground_truth_seismic_copy[0] = scaler1.transform(ground_truth_seismic_copy[0].reshape(-1, 1)).reshape(96, 32, 32)*2-1
ground_truth_seismic_copy[1] = scaler2.transform(ground_truth_seismic_copy[1].reshape(-1, 1)).reshape(96, 32, 32)*2-1
ground_truth_seismic_copy[2] = scaler3.transform(ground_truth_seismic_copy[2].reshape(-1, 1)).reshape(96, 32, 32)*2-1
ground_truth_seismic_copy = ground_truth_seismic_copy.reshape(1, 3, 96, 32, 32)
ground_truth_seismic_copy = torch.tensor(ground_truth_seismic_copy.astype(np.float32), device = 'cuda').reshape(1, 3, 96, 32, 32)
ground_truth_seismic_copy.to(device)

for i in range(N_real):
    random_z = Variable(torch.randn(1, 8).type(torch.cuda.FloatTensor))
    out = np.array(generator(ground_truth_seismic_copy, random_z).detach().cpu())
    out = upscale_3d(out)
    print(f'{i}# prediction model is generated')

    out[:, 1] = (out[:, 1] + 1) / 2
    out[:, 1] = np.where(out[:, 1] > 0.5, 1, 0)
    out[:, 0] = cdf_mapping_by_facies(out[:, 0], cdf_model[:, 0], out[:, 1], cdf_model[:, 1])
    
    ground_truth_pred[i] = out[0]

ground_truth_pred = np.concatenate([ground_truth_pred, por_to_perm(ground_truth_pred[:, 0], ground_truth_pred[:, 1]).reshape(N_real, 1, 16, 32, 32)], axis=1)

In [ ]:
ground_truth = np.concatenate([ground_truth, por_to_perm(ground_truth[:, 0], ground_truth[:, 1]).reshape(1, 1, 16, 32, 32)], axis=1)

In [ ]:
avg_ground_truth_pred = ground_truth_pred.mean(axis=0)
# Plot ensemble model & prediction model
N_real = 0
plotter = pv.Plotter(shape = (2, 3), window_size = [1024, 1024], border=False)

plotter.subplot(0, 0)
visual_grid_1 = ground_truth[0, 1]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_1[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_1[::-1].T.flatten(order="F")  # Flatten the array
slice = grid.slice_orthogonal(x=9, y=9, z=2)
plotter.add_mesh(grid, show_scalar_bar=False)
plotter.add_title('True', font_size=12)

plotter.subplot(0, 1)
visual_grid_2 = ground_truth[0, 0]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_2[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_2[::-1].T.flatten(order="F")  # Flatten the array
plotter.add_mesh(grid, show_scalar_bar=True, scalar_bar_args={'title':'Porosity [fraction]', 'title_font_size':15,
                                                            'label_font_size':10})
plotter.add_title('Prediction', font_size=12)
plotter.update_scalar_bar_range([0, 0.35])

plotter.subplot(0, 2)
visual_grid_1 = ground_truth[0, 2]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_1[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_1[::-1].T.flatten(order="F")  # Flatten the array
# slice = grid.slice_orthogonal(x=9, y=9, z=2)
plotter.add_mesh(grid, show_scalar_bar=True, cmap='jet', scalar_bar_args={'title':'Permeability [mD]', 'title_font_size':15,
                                                            'label_font_size':10})
plotter.update_scalar_bar_range([0, 150])


plotter.subplot(1, 0)
visual_grid_2 = avg_ground_truth_pred[1]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_2[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_2[::-1].T.flatten(order="F")  # Flatten the array
# slice = grid.slice_orthogonal(x=9, y=9, z=2)
plotter.add_mesh(grid, show_scalar_bar=False)

plotter.subplot(1, 1)
visual_grid_2 = avg_ground_truth_pred[0]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_2[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_2[::-1].T.flatten(order="F")  # Flatten the array
# slice = grid.slice_orthogonal(x=9, y=9, z=2)
plotter.add_mesh(grid, show_scalar_bar=False)
plotter.update_scalar_bar_range([0, 0.35])

plotter.subplot(1, 2)
visual_grid_2 = avg_ground_truth_pred[2]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_2[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_2[::-1].T.flatten(order="F")  # Flatten the array
# slice = grid.slice_orthogonal(x=9, y=9, z=2)
plotter.add_mesh(grid, show_scalar_bar=False, cmap='jet')
plotter.update_scalar_bar_range([0, 150])

plotter.show()

In [ ]:
plotter = pv.Plotter(shape = (4, 4), window_size = [1024, 1024], border=False)

for i in range(16):
    plotter.subplot(i//4, i%4)
    visual_grid_2 = ground_truth_pred[i, 1]
    grid = pv.ImageData()
    grid.dimensions = np.array(visual_grid_2[::-1].T.shape) + 1
    grid.origin = (1, 1, 1)  # The bottom left corner of the data set
    grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
    grid.cell_data["values"] = visual_grid_2[::-1].T.flatten(order="F")  # Flatten the array
    plotter.add_mesh(grid, show_scalar_bar=False)
plotter.show()

### **Step 4**: PCA analysis

In [ ]:
PCA_model = np.ones((4000, 2, 16, 32, 32))
PCA_model[:, 0] = Porosity
PCA_model[:, 1] = Facies

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2) # 주성분을 몇개로 할지 결정
printcipalComponents = pca.fit_transform(PCA_model.reshape(4000, -1))

In [ ]:
color = [[10*i]*40 for i in range(10)]
color = np.repeat(np.array(color).reshape(1, 10, 40), 10, axis=0).reshape(1, -1)

In [ ]:
ref_num = 20
plt.figure(figsize=(6, 5))

plt.scatter(printcipalComponents[ref_num, 0], printcipalComponents[ref_num, 1], c='None', edgecolors='r', marker='^', s=50, alpha=.7, label='Reference', zorder=2)
plt.scatter(printcipalComponents[:, 0], printcipalComponents[:, 1], c=color, s=3, cmap='grey', alpha=.7, label='Initial ensemble', zorder=1)
plt.xlabel('1st Printcipal Component')
plt.ylabel('2nd Printcipal Component')
plt.legend()

plt.colorbar()

In [ ]:
prediction_pca = pca.transform(ground_truth_pred[:, :2].reshape(100, -1))

In [ ]:
plt.figure(figsize=(5, 5))

plt.scatter(printcipalComponents[0, 0], printcipalComponents[0, 1], c='None', edgecolors='r', marker='^', s=50, alpha=.7, label='Reference')
plt.scatter(printcipalComponents[:, 0], printcipalComponents[:, 1], c='grey', s=3, alpha=.3, label='Initial ensemble')
plt.scatter(prediction_pca[:, 0], prediction_pca[:, 1], c='b', s=3, alpha=.8, label='prediction')
plt.xlabel('1st Printcipal Component')
plt.ylabel('2nd Printcipal Component')
plt.legend()

### **Step 5**: Comparison of dynamic property (gas saturation)

In [ ]:
NX, NY, NZ = 32, 32, 16
# instantiate the simulation object
sim_ccs = Sim_CCS()

# set the simulation parameters
input_parameters = {'porosity': ground_truth_pred[:, 0],
                    'permeability': ground_truth_pred[:, 2],
                    'Facies': ground_truth_pred[:, 1]}

# set the bottom hole pressure constraint
maximum_bhp = 3500 # psi
sim_ccs.reset_bhp_constraint(maximum_bhp)

# set the injection amount
injection_rate = 15_839_619 # scf/day
sim_ccs.reset_injection_amout(injection_rate)

# set your CMG exe path
cmg_exe = '"C:\\Program Files\\CMG\\GEM\\2024.20\\Win_x64\\EXE\\gm202420.exe"'
sim_ccs.reset_parameters_to_play(list(input_parameters.keys()))
sim_ccs.reset_CMG_exe(cmg_exe)

In [ ]:
# generate the simulation cases of the models generated by BicycleGAN
sim_ccs.ensemble_simulation(sub_dir='ground_truth_pred\\pred' , input_parameters=input_parameters)

# TODO: Template data 시뮬 시간을 7년으로 연장

In [ ]:
# Run the simulation with multiprocessing
sim_ccs.run_multiple_CMG(N_thread=10, sub_dir = os.path.join(os.getcwd(), 'ground_truth_pred\\pred'))

In [ ]:
# Extract gas saturation from results
sim_ccs.reset_Ptime([2025, 2026, 2027, 2028, 2029, 2030, 2031])

ground_truth_pred_Sg = np.ones((100, 8, 16, 32, 32)) * -9999
for i in range(100):
    ground_truth_pred_Sg[i] =  sim_ccs.read_grid_results(run_dir=f'ground_truth_pred\\pred_{i}')['GasSaturation']

ground_truth_pred_Sg_mean = ground_truth_pred_Sg.mean(axis=0)
ground_truth_pred_Sg_std = ground_truth_pred_Sg.std(axis=0)

In [ ]:
# Plot ensemble model & prediction model
plotter = pv.Plotter(shape = (2, 2), window_size = [1024, 1024], border=False)

plotter.subplot(0, 0)
visual_grid_1 = ground_truth_pred_Sg_mean[4]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_1[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_1[::-1].T.flatten(order="F")  # Flatten the array
slice = grid.slice_orthogonal(x=9, y=9, z=2)
plotter.add_mesh(slice, show_scalar_bar=True, cmap='binary', scalar_bar_args={'title':'Gas saturation', 'title_font_size':15,
                                                            'label_font_size':10, 'position_x': 0.1,
                                                            'width':0.3})
plotter.add_title('Gas saturation after CO2 injection 5 years', font_size=12)

plotter.subplot(0, 1)
visual_grid_2 = ground_truth_pred_Sg_mean[-1]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_2[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_2[::-1].T.flatten(order="F")  # Flatten the array
slice = grid.slice_orthogonal(x=9, y=9, z=2)
plotter.add_mesh(slice, show_scalar_bar=False, cmap='binary')

plotter.add_title('Gas saturation after CO2 injection 7 years', font_size=12)

plotter.subplot(1, 0)
visual_grid_1 = ground_truth_pred_Sg_std[4]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_1[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_1[::-1].T.flatten(order="F")  # Flatten the array
slice = grid.slice_orthogonal(x=9, y=9, z=2)
plotter.add_mesh(slice, show_scalar_bar=True, cmap='jet', scalar_bar_args={'title':'std in gas saturation', 'title_font_size':15,
                                                            'label_font_size':10})

plotter.subplot(1, 1)
visual_grid_1 = ground_truth_pred_Sg_std[-1]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_1[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_1[::-1].T.flatten(order="F")  # Flatten the array
slice = grid.slice_orthogonal(x=9, y=9, z=2)
plotter.add_mesh(slice, show_scalar_bar=True, cmap='jet', scalar_bar_args={'title':'std in gas saturation', 'title_font_size':15,
                                                            'label_font_size':10})

plotter.show()

### **Step 6**: Signal_to_noise stress analysis

In [ ]:
SNR_list = [30, 50, 80]
ground_truth_seismic_with_noise = np.ones((len(SNR_list), 3, 96, 32, 32))
for _, SNR in enumerate(SNR_list):
    ground_truth_seismic_with_noise[_, 0] = add_noise_to_seismic(ground_truth_seismic[0], SNR)
    ground_truth_seismic_with_noise[_, 1] = add_noise_to_seismic(ground_truth_seismic[1], SNR)
    ground_truth_seismic_with_noise[_, 2] = ground_truth_seismic_with_noise[_, 1] - ground_truth_seismic_with_noise[_, 0]
    print(f'{SNR} case is done!')

In [ ]:
plotter = pv.Plotter(shape = (3, 4), window_size = [1024, 1024], border=False)

plotter.subplot(0, 0)
visual_grid_1 = ground_truth_seismic[0]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_1[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1/6)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_1[::-1].T.flatten(order="F")  # Flatten the array
slice = grid.slice_orthogonal(x=9, y=9, z=2)
plotter.add_mesh(slice, show_scalar_bar=False, cmap='seismic')
plotter.add_title('SNR: 1.0', font_size=12)
plotter.update_scalar_bar_range([-0.4, 0.4])

plotter.subplot(0, 1)
visual_grid_2 = ground_truth_seismic_with_noise[2, 0]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_2[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1/6)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_2[::-1].T.flatten(order="F")  # Flatten the array
slice = grid.slice_orthogonal(x=9, y=9, z=2)
plotter.add_mesh(slice, show_scalar_bar=False, cmap='seismic')
plotter.add_title('SNR: 0.8', font_size=12)
plotter.update_scalar_bar_range([-0.4, 0.4])

plotter.subplot(0, 2)
visual_grid_1 = ground_truth_seismic_with_noise[1, 0]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_1[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1/6)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_1[::-1].T.flatten(order="F")  # Flatten the array
slice = grid.slice_orthogonal(x=9, y=9, z=2)
plotter.add_mesh(slice, show_scalar_bar=False, cmap='seismic')
plotter.add_title('SNR: 0.5', font_size=12)
plotter.update_scalar_bar_range([-0.4, 0.4])

plotter.subplot(0, 3)
visual_grid_1 = ground_truth_seismic_with_noise[0, 0]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_1[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1/6)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_1[::-1].T.flatten(order="F")  # Flatten the array
slice = grid.slice_orthogonal(x=9, y=9, z=2)
plotter.add_mesh(slice, show_scalar_bar=False, cmap='seismic')
plotter.add_title('SNR: 0.3', font_size=12)
plotter.update_scalar_bar_range([-0.4, 0.4])


plotter.subplot(1, 0)
visual_grid_1 = ground_truth_seismic[1]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_1[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1/6)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_1[::-1].T.flatten(order="F")  # Flatten the array
slice = grid.slice_orthogonal(x=9, y=9, z=2)
plotter.add_mesh(slice, show_scalar_bar=False, cmap='seismic')
plotter.update_scalar_bar_range([-0.4, 0.4])

plotter.subplot(1, 1)
visual_grid_2 = ground_truth_seismic_with_noise[2, 1]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_2[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1/6)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_2[::-1].T.flatten(order="F")  # Flatten the array
slice = grid.slice_orthogonal(x=9, y=9, z=2)
plotter.add_mesh(slice, show_scalar_bar=False, cmap='seismic')
plotter.update_scalar_bar_range([-0.4, 0.4])

plotter.subplot(1, 2)
visual_grid_1 = ground_truth_seismic_with_noise[1, 1]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_1[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1/6)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_1[::-1].T.flatten(order="F")  # Flatten the array
slice = grid.slice_orthogonal(x=9, y=9, z=2)
plotter.add_mesh(slice, show_scalar_bar=False, cmap='seismic')
plotter.update_scalar_bar_range([-0.4, 0.4])

plotter.subplot(1, 3)
visual_grid_1 = ground_truth_seismic_with_noise[0, 1]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_1[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1/6)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_1[::-1].T.flatten(order="F")  # Flatten the array
slice = grid.slice_orthogonal(x=9, y=9, z=2)
plotter.add_mesh(slice, show_scalar_bar=False, cmap='seismic')
plotter.update_scalar_bar_range([-0.4, 0.4])

plotter.subplot(2, 0)
visual_grid_1 = ground_truth_seismic[2]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_1[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1/6)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_1[::-1].T.flatten(order="F")  # Flatten the array
slice = grid.slice_orthogonal(x=9, y=9, z=2)
plotter.add_mesh(slice, show_scalar_bar=False, cmap='seismic')
plotter.update_scalar_bar_range([-0.4, 0.4])

plotter.subplot(2, 1)
visual_grid_2 = ground_truth_seismic_with_noise[2, 2]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_2[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1/6)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_2[::-1].T.flatten(order="F")  # Flatten the array
slice = grid.slice_orthogonal(x=9, y=9, z=2)
plotter.add_mesh(slice, show_scalar_bar=False, cmap='seismic')
plotter.update_scalar_bar_range([-0.4, 0.4])

plotter.subplot(2, 2)
visual_grid_1 = ground_truth_seismic_with_noise[1, 2]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_1[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1/6)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_1[::-1].T.flatten(order="F")  # Flatten the array
slice = grid.slice_orthogonal(x=9, y=9, z=2)
plotter.add_mesh(slice, show_scalar_bar=False, cmap='seismic')
plotter.update_scalar_bar_range([-0.4, 0.4])

plotter.subplot(2, 3)
visual_grid_1 = ground_truth_seismic_with_noise[0, 2]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_1[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1/6)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_1[::-1].T.flatten(order="F")  # Flatten the array
slice = grid.slice_orthogonal(x=9, y=9, z=2)
plotter.add_mesh(slice, show_scalar_bar=False, cmap='seismic')
plotter.update_scalar_bar_range([-0.4, 0.4])

plotter.show()

In [ ]:
ground_truth_seismic_with_noise.shape

In [ ]:
N_real = 100
ground_truth_with_noise_pred = np.ones((len(SNR_list), N_real, 2, 16, 32, 32)) *-9999

for _, SNR in enumerate(SNR_list):
    ground_truth_seismic_copy = ground_truth_seismic_with_noise[_].copy()
    ground_truth_seismic_copy[0] = scaler1.transform(ground_truth_seismic_copy[0].reshape(-1, 1)).reshape(96, 32, 32)*2-1
    ground_truth_seismic_copy[1] = scaler2.transform(ground_truth_seismic_copy[1].reshape(-1, 1)).reshape(96, 32, 32)*2-1
    ground_truth_seismic_copy[2] = scaler3.transform(ground_truth_seismic_copy[2].reshape(-1, 1)).reshape(96, 32, 32)*2-1
    ground_truth_seismic_copy = ground_truth_seismic_copy.reshape(1, 3, 96, 32, 32)
    ground_truth_seismic_copy = torch.tensor(ground_truth_seismic_copy.astype(np.float32), device = 'cuda').reshape(1, 3, 96, 32, 32)
    ground_truth_seismic_copy.to(device)

    for i in range(N_real):
        random_z = Variable(torch.randn(1, 8).type(torch.cuda.FloatTensor))
        out = np.array(generator(ground_truth_seismic_copy, random_z).detach().cpu())
        out = upscale_3d(out)
        print(f'{i}# prediction model of {SNR} SNR ratio is generated')

        out[:, 1] = (out[:, 1] + 1) / 2
        out[:, 1] = np.where(out[:, 1] > 0.5, 1, 0)
        out[:, 0] = cdf_mapping_by_facies(out[:, 0], cdf_model[:, 0], out[:, 1], cdf_model[:, 1])
        
        ground_truth_with_noise_pred[_, i] = out[0]

# ground_truth_pred = np.concatenate([ground_truth_pred, por_to_perm(ground_truth_pred[:, :, ], ground_truth_pred[:, 1]).reshape(N_real, 1, 16, 32, 32)], axis=1)
avg_ground_truth_with_noise_pred = ground_truth_with_noise_pred.mean(axis=1)
std_ground_truth_with_noise_pred = ground_truth_with_noise_pred.std(axis=1)

In [ ]:
# Plot average of prediction model with SNR
plotter = pv.Plotter(shape = (2, 4), window_size = [1024, 1024], border=False)


'''------------------------------Avg.facies plot------------------------------'''
plotter.subplot(0, 0)
visual_grid_1 = ground_truth_pred.mean(axis=0)[1]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_1[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_1[::-1].T.flatten(order="F")  # Flatten the array
slice = grid.slice_orthogonal(x=9, y=9, z=2)
plotter.add_mesh(grid, show_scalar_bar=False)
plotter.add_title('SNR: 1.0', font_size=12)

plotter.subplot(0, 1)
visual_grid_2 = avg_ground_truth_with_noise_pred[0, 1]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_2[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_2[::-1].T.flatten(order="F")  # Flatten the array
plotter.add_mesh(grid, show_scalar_bar=False)
plotter.add_title('SNR: 0.8', font_size=12)

plotter.subplot(0, 2)
visual_grid_1 = avg_ground_truth_with_noise_pred[1, 1]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_1[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_1[::-1].T.flatten(order="F")  # Flatten the array
# slice = grid.slice_orthogonal(x=9, y=9, z=2)
plotter.add_mesh(grid, show_scalar_bar=False)
plotter.add_title('SNR: 0.5', font_size=12)

plotter.subplot(0, 3)
visual_grid_1 = avg_ground_truth_with_noise_pred[2, 1]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_1[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_1[::-1].T.flatten(order="F")  # Flatten the array
plotter.add_mesh(grid, show_scalar_bar=False)
plotter.add_title('SNR: 0.3', font_size=12)

'''------------------------------Avg.porosity plot------------------------------'''
plotter.subplot(1, 0)
visual_grid_1 = ground_truth_pred.mean(axis=0)[0]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_1[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_1[::-1].T.flatten(order="F")  # Flatten the array
slice = grid.slice_orthogonal(x=9, y=9, z=2)
plotter.add_mesh(grid, show_scalar_bar=False)
plotter.update_scalar_bar_range([0, 0.35])

plotter.subplot(1, 1)
visual_grid_2 = avg_ground_truth_with_noise_pred[0, 0]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_2[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_2[::-1].T.flatten(order="F")  # Flatten the array
plotter.add_mesh(grid, show_scalar_bar=False)
plotter.update_scalar_bar_range([0, 0.35])

plotter.subplot(1, 2)
visual_grid_1 = avg_ground_truth_with_noise_pred[1, 0]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_1[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_1[::-1].T.flatten(order="F")  # Flatten the array
plotter.add_mesh(grid, show_scalar_bar=False)
plotter.update_scalar_bar_range([0, 0.35])

plotter.subplot(1, 3)
visual_grid_1 = avg_ground_truth_with_noise_pred[2, 0]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_1[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_1[::-1].T.flatten(order="F")  # Flatten the array
plotter.add_mesh(grid, show_scalar_bar=False)
plotter.update_scalar_bar_range([0, 0.35])

plotter.show()

In [ ]:
# Plot average of prediction model with SNR
plotter = pv.Plotter(shape = (2, 4), window_size = [1024, 1024], border=False)


'''------------------------------Avg.facies plot------------------------------'''
plotter.subplot(0, 0)
visual_grid_1 = ground_truth_pred.std(axis=0)[1]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_1[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_1[::-1].T.flatten(order="F")  # Flatten the array
slice = grid.slice_orthogonal(x=9, y=9, z=2)
plotter.add_mesh(grid, cmap='jet', show_scalar_bar=False)
plotter.add_title('SNR: 1.0', font_size=12)

plotter.subplot(0, 1)
visual_grid_2 = std_ground_truth_with_noise_pred[0, 1]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_2[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_2[::-1].T.flatten(order="F")  # Flatten the array
plotter.add_mesh(grid, cmap='jet', show_scalar_bar=False)
plotter.add_title('SNR: 0.8', font_size=12)

plotter.subplot(0, 2)
visual_grid_1 = std_ground_truth_with_noise_pred[1, 1]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_1[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_1[::-1].T.flatten(order="F")  # Flatten the array
# slice = grid.slice_orthogonal(x=9, y=9, z=2)
plotter.add_mesh(grid, cmap='jet', show_scalar_bar=False)
plotter.add_title('SNR: 0.5', font_size=12)

plotter.subplot(0, 3)
visual_grid_1 = std_ground_truth_with_noise_pred[2, 1]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_1[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_1[::-1].T.flatten(order="F")  # Flatten the array
plotter.add_mesh(grid, cmap='jet', show_scalar_bar=False)
plotter.add_title('SNR: 0.3', font_size=12)

'''------------------------------Avg.porosity plot------------------------------'''
plotter.subplot(1, 0)
visual_grid_1 = ground_truth_pred.std(axis=0)[0]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_1[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_1[::-1].T.flatten(order="F")  # Flatten the array
slice = grid.slice_orthogonal(x=9, y=9, z=2)
plotter.add_mesh(grid, cmap='jet', show_scalar_bar=False)

plotter.subplot(1, 1)
visual_grid_2 = std_ground_truth_with_noise_pred[0, 0]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_2[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_2[::-1].T.flatten(order="F")  # Flatten the array
plotter.add_mesh(grid, cmap='jet', show_scalar_bar=False)

plotter.subplot(1, 2)
visual_grid_1 = std_ground_truth_with_noise_pred[1, 0]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_1[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_1[::-1].T.flatten(order="F")  # Flatten the array
plotter.add_mesh(grid, cmap='jet', show_scalar_bar=False)

plotter.subplot(1, 3)
visual_grid_1 = std_ground_truth_with_noise_pred[2, 0]
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid_1[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid_1[::-1].T.flatten(order="F")  # Flatten the array
plotter.add_mesh(grid, cmap='jet', show_scalar_bar=False)

plotter.show()

In [ ]:
for _, SNR in enumerate(SNR_list):
    plt.hist(std_ground_truth_with_noise_pred[_, 0].flatten(), bins=30, density=True, alpha=.6, label=f'SNR: {SNR}')
plt.legend()

### **APPENDIX**: Loss curve check!

In [ ]:
f = open(os.path.join(os.getcwd(), 'runs', 'July_20_2024_12_56AM_training_log.json'))
data = json.load(f)

In [ ]:
plt.figure(figsize=(5, 5))

for key in ['discriminator_loss', 'generator_alone_loss']:
    epochs = [int(epoch) for epoch in data[key].keys()]
    values = [float(value) for value in data[key].values()]

    plt.plot(epochs, values, label=f'{key}')

plt.xlabel('Epoch', fontdict={'fontsize':12})
plt.ylabel('Loss', fontdict={'fontsize':12})
plt.legend()

In [ ]:
plt.figure(figsize=(5, 5))

for key in ['generator_loss', 'EG_loss']:
    epochs = [int(epoch) for epoch in data[key].keys()]
    values = [float(value) for value in data[key].values()]

    plt.plot(epochs, values, label=f'{key}')

plt.xlabel('Epoch', fontdict={'fontsize':12})
plt.ylabel('Loss', fontdict={'fontsize':12})
plt.legend()